<img src="/static/base/images/logo.png?v=641991992878ee24c6f3826e81054a0f" alt="Jupyter Notebook">
<h1 style="text-align: center">Notebook 7 - Python Flask</h1>

<h3>Prerequisites</h3>

- You must have Python 3 installed on your system (<a href="https://www.python.org/downloads/">Download</a>)
- You must have Jupyter installed on your system (<a href="https://jupyter.org/install">Download</a>)
- Some knowledge of Python, HTML, Javascript and SQL may be required

<h3>Explanation of Notebook 7</h3>

This notebook will involve building services that interact with a database; you will create a simple table using SQLite3 for entering requests for stocks, and you'll develop methods for this table that allows you to create, update and delete rows using the Flask framework. This will also involve the use of HTML for the web pages for the application and the "requests" library to retrieve submitted data through those pages.

+ SQLite3 is a module included in Python (2.5+), SQLite is an embedded relational database engine, and this module is used to create a database that's stored in a single disk file, meaning that you do not have to create a server to access it.<br>
+ Flask on the other hand is a framework that allows you to work with SQLite3 module to make a CRUD (create, retrieve, update, delete) application.

The table will include the following properties/columns: 
+ <code>asset</code> 
+ <code>order_date</code>
+ <code>amount</code>
+ <code>price</code>

<h3>Getting started</h3>

As mentioned above, "SQLite3" will be preinstalled with Python (2.5+), however, you'll need to install "Flask" and "reqests" manually. <br>
You can do this using pip in Command Prompt (Windows), or the Terminal (MacOS/Linux):<br>
<code>pip install Flask</code><br>
<code>pip install requests</code>

Otherwise, run the cells below.

In [ ]:
pip install Flask

In [ ]:
pip install requests

After doing so, you may proceed with the notebook; make sure that your kernel is set to Python 3 as we won't be working with R.

<h3>Creating the database & table</h3>

In [ ]:
import sqlite3

DB = sqlite3.connect("data/database.db")
c = DB.cursor()

The above cell will create a database named "database.db" on the folder "data".<br>
The 'c' object is assigned to the database cursor, and the reason why we need it is so that we can execute SQL statements onto that database using that cursor, such as making new tables like shown below.

In [ ]:
def create_table():
    c.execute("CREATE TABLE IF NOT EXISTS stockTable(id INTEGER PRIMARY KEY AUTOINCREMENT, asset TEXT, order_date TEXT, amount INTEGER, price REAL)")
    print("Table created") #To ensure it worked
    c.close()

The above "create_table" function will execute an SQL command to create a table named "stockTable" with four columns; id, asset, order_date, amount, and price, the data type "REAL" refers to floats, which means it allows decimal points.

The reason why a column for "id" was made was to ensure that each share is uniquely identified as it is used as the primary key for the table.

This will be used to store all the shares which can be managed from the application, which is created in the following cells.

<h3>Example</h3>

<b>Warning: Make sure you restart the kernel if you run this example to run the rest of the code</b>

In [ ]:
from flask import *  

app = Flask(__name__) #Create Flask application

#When in root directory
@app.route("/")
def index():
    return render_template("index.html");  #Go to index.html

if (__name__ == '__main__'):
    create_table() #Create the table
    app.run(host='127.0.0.1', port=5000)

The line mentioning the route of the application refers to what needs to be handled when the you visit localhost on port 5000, which in this case, it should go to the function named "index", this process is known as routing.

If you run the above cell, it will create a local server and look for a file name "index.html" using the "render_template" function, this will look for that file in a folder named "templates" (It must be called this as otherwise it'd not work), this is because Flask is run on a website, and since the first function that is ran is "main", it will check if the current function is on "main" which if it is it runs the application.

You can look at the html files and modify them in the <a href="http://localhost:8888/lab/tree/Notebook%207/templates">templates</a> folder, the "index.html" file looks like this:

You can see that it has three articles, and each of them has a "href" tag linking to each route, and when the user clicks one of them, the application detects this.

Something to note is that in Jupyter, running a Flask web application will be continuous and thus you won't be able to run anything else until you restart the kernel with cleared outputs, due to this, the code for running the application will be in a separate cell, check the cell below.

<h3>Index and add share pages</h3>

In [ ]:
from flask import *

app = Flask(__name__) #Create Flask application

#When in root directory
@app.route("/")
def index():
    return render_template("index.html");  #Go to index.html 

#When in 'add' directory
@app.route("/add")  
def add():  
    return render_template("add.html")  #Go to add.html

This cell creates two routes, for the "index" and the "add" pages; index will be used as the homepage while the add page will be used to insert shares into the database from the web application, but we have to make sure that data the user added gets added into the database, and so we'll need to make a new route for that.

<h3>Adding rows</h3>

In [ ]:
#When in 'savedetails' directory
@app.route("/savedetails",methods = ["POST"]) 
def saveDetails():  
    msg = "msg"
    if (request.method == "POST"): #If the user submits data
        try:  
            asset = request.form["asset"]  #Get asset
            order_date = request.form["order_date"]  #Get order date
            amount = request.form["amount"]  #Get amount
            price = request.form["price"]  #Get price
            with sqlite3.connect("data/database.db") as con: #Connect to database
                cur = con.cursor()
                cur.execute("INSERT into stockTable (asset, order_date, amount, price) values (?,?,?,?)",(asset,order_date,amount,price)) #Add the user's data  
                con.commit()  
                msg = "Share successfully added"
        except:  
            con.rollback()  
            msg = "We can not add the share to the list"  
        finally:  
            return render_template("success.html",msg = msg)  #Go to the successful/success.html page with the message
            con.close()  

You can see that there are variables for each value (asset, order_date, amount and price), and it connects to the database once again so that it can add those values, and if something fails it will rollback any changes made. Regardless of the success of the user's action, it will go to "success.html" page where it'll display a message either saying "Share successfully added" or "We can not add the share to the list", you can see that it does this using the second parameter in the "render_template" function at the end, to make things easier, the cell below shows the "success.html" file.

The "h3" tag catches the "msg" parameter's value from the route, this is because the enclosing curly brackets of the word "msg" in the html file acts as a placeholder for that parameter value, and so upon viewing that page, it will display text based on the success onto that "h3" tag.

You'll notice something different with this route; it has another parameter for the "methods", which is set to "POST" in this case. It is what is used for gaining submitted data when going to the specified web page, which in this case is the file "add.html", which is shown below.

There's an input for all 4 variables (asset, order_date, amount and price), and when you click submit, the action is "/savedetails" with the method "post", this means it will send the form with those variables onto that "/savedetails" page which will be caught in the route like shown above.

You can also see that there's Javascript code where it gets the current date and time, this is so that it is automatically taken and set to the share date so that the user wouldn't make a mistake, hence why it is a read-only field.

<h3>Viewing rows</h3>

We need to also make sure that these values are added onto the table in the database, so we need to make a new route to view all the shares as a table.

In [ ]:
#When in 'view' directory
@app.route("/view")  
def view():  
    con = sqlite3.connect("data/database.db") #Connect to database
    con.row_factory = sqlite3.Row #Converts the plain tuple to a more useful object
    cur = con.cursor()  
    cur.execute("select * from stockTable") #Select everything in the table
    rows = cur.fetchall() #Get all the rows
    return render_template("view.html",rows = rows)  #Go to view.html with the rows

This time, you can see that after it connects to a database it runs an SQL statement to select all the data from the table and gets all of its rows, these rows are passed on as a parameter onto the "view.html" page so that it can be displayed, the following cell will show what the "view.html" file looks like.

You can see that this time it's slightly difference, this is because not only do we receive the "rows" parameter but also use it as a for loop so that it iterates through each row in those rows and creates a table row for each with the values of those rows, and you can also notice that there are placeholders for the 'value' property of each input, which are filled when being directed to this page with the parameters, but what if we wanted to delete some of these rows? Check the cell below.

<h3>Deleting rows</h3>

In [ ]:
#When in 'delete' directory
@app.route("/delete")  
def delete():  
    con = sqlite3.connect("data/database.db") #Connect to database
    con.row_factory = sqlite3.Row #Converts the plain tuple to a more useful object
    cur = con.cursor()  
    cur.execute("select * from stockTable") #Select everything in the table
    rows = cur.fetchall() #Get all the rows
    return render_template("delete.html",rows = rows)  
 
#When in 'deleterecord' directory
@app.route("/deleterecord", methods = ["POST"])  
def deleterecord():  
    id = request.form["id"] #Get id the user entered
    with sqlite3.connect("data/database.db") as con: #Connect to database
        try:  
            cur = con.cursor()  
            cur.execute("delete from stockTable where id = ?",id) #Delete the share containing the id  
            msg = "Share successfully deleted"  
        except:  
            con.rollback() 
            msg = "Share can't be deleted"  
        finally:  
            return render_template("success.html",msg = msg) #Go to successful/success.html page with the message
            con.close()

You'll see that there's two routes created, one for the page to delete ("delete.html") and the other to execute the deletion ("delete.html/deleterecord -> delete_record.html"), you can see that it uses the "POST" method once again but this time to only get the "id" variable, which uniquely identifies each row and thus making retrieving and deleting rows much easier, when a deletion is successful the message will be "Share successfully deleted", otherwise "Share can't be deleted", both of which are also passed through the "msg" parameter, the "delete.html" file is shown below.

As you can see, it's similar to the "add.html" file, but with only one input, and it goes to "/deleterecord" instead, which gets caught in the route above and deletes the share with the entered ID, however, the portion for the table of rows is also implemented, this is so that you can check what is changed/deleted without having to go to another page, but what if we wanted to update a share instead of deleting one? Check the cell below.

<h3>Updating rows</h3>

In [ ]:
#When in 'getdetails' directory
@app.route("/update", methods = ["POST"])  
def update():  
    con = sqlite3.connect("data/database.db") #Connect to database
    con.row_factory = sqlite3.Row #Converts the plain tuple to a more useful object
    cur = con.cursor()  
    
    id = request.form["id"] #Get ID
    
    newRows = [0, 0, 0, 0]
    userRow = [0, 0, 0, 0]
    msg = "msg"
    
    if (request.form['submit'] == "Get details"): #If "Get details" button is pressed
        try:  
            cur.execute("select * from stockTable where id = ?",id) #Select all the values in the table with the rows with the id
            rows = cur.fetchall() #Get all rows (Will only return 1)
            userRow = rows[0] #Get first row
            
            #Update the rows
            cur.execute("select * from stockTable") #Select everything in the table
            newRows = cur.fetchall() #Get all the rows
            msg = "Share successfully retrieved"  
        except:  
            con.rollback() 
            msg = "Share doesn't exist"  
        finally:  
            #Pass all the column values
            return render_template("view.html", id = id, asset = userRow["asset"], date = userRow["order_date"], amount = userRow["amount"], price = userRow["price"], rows = newRows, msg = msg)       
            con.close()
    else: #If "Update" button is pressed
        try:         
            asset = request.form["asset"] #Get entered asset
            amount = request.form["amount"] #Get entered amount
            price = request.form["price"] #Get entered price

            cur.execute("update stockTable set asset=?, amount=?, price=? where id = ?", (asset, amount, price, id)) #Update the values in the tables

            #Update the rows
            cur.execute("select * from stockTable") #Select everything in the table
            newRows = cur.fetchall() #Get all the rows
            msg = "Share successfully updated" 
        except:  
            con.rollback() 
            msg = "Share can't be updated"  
        finally:
            return render_template("view.html", rows = newRows, msg = msg)  #Go to view.html with the rows
            con.close()

This will create a route for the page for updating the shares, you can see that there's two different paths; one for the submit button and the other for the button to get the details, it all works around the unique number given to each row, you can see that the object "userRow" is used to store the row for the id the user entered, which gets passed down in divisons in terms of columns whereas in the second portion you simply just grab the details from what the user entered into the form, which can be used to update the table with the new data.

Either way, some error handling is made to ensure that the user wouldn't make as many mistakes, although it isn't fully implemented and so relying on it isn't the best, however it is there just as a simple example of how this works. The same page, "view.html" is used for this as well, this is so that you can update the values while looking at the list instead of having to go back and forth.<br>

<h3>Running the web application</h3>

Finally, we can now run this application using the method below.

In [ ]:
if (__name__ == "__main__"):  
    create_table()
    app.run(host='127.0.0.1', port=5000)

Copy and paste the shown URL after running it and paste it in a new tab to use the application.

<h3>The end</h3>

This concludes the end of notebook 7, you've explored the Flask framework as well as using request methods to pass in user information through web applications where you managed a database with a table using SQL in regards to shares.